In [242]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.3 MB/s  0:00:00


In [244]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor

In [221]:
dir_data = os.path.join("data", "processed")

path_train = os.path.join(dir_data, "train.csv")
path_val = os.path.join(dir_data, "validation.csv")
path_test = os.path.join(dir_data, "test.csv")

In [222]:
df_train = pd.read_csv(path_train)
df_val = pd.read_csv(path_val)
df_test = pd.read_csv(path_test)

In [223]:
df_train["mean_violent_prev_3mo"] = df_train[["violent_lag_1", "violent_lag_2",	"violent_lag_3"]].mean(axis=1)
df_val["mean_violent_prev_3mo"] = df_val[["violent_lag_1", "violent_lag_2",	"violent_lag_3"]].mean(axis=1)
df_test["mean_violent_prev_3mo"] = df_test[["violent_lag_1", "violent_lag_2", "violent_lag_3"]].mean(axis=1)

#df_train["log_median_household_income"] = np.log(df_train["median_household_income"])
#df_val["log_median_household_income"] = np.log(df_val["median_household_income"])
#df_test["log_median_household_income"] = np.log(df_test["median_household_income"])

#df_train["log_pop_total"] = np.log1p(df_train["pop_total"])
#df_val["log_pop_total"] = np.log1p(df_val["pop_total"])
#df_test["log_pop_total"] = np.log1p(df_test["pop_total"])

In [224]:
df_train

,Unnamed: 0,ZCTA,incident_month_timestamp,crime_Non-Violent,crime_Violent,violent_lag_1,violent_lag_2,violent_lag_3,season,month_sin,month_cos,pop_total,median_household_income,gini_index,unemployment_rate,poverty_rate,mean_violent_prev_3mo
0,0,90001,2020-04-01,16.0,25.0,16.0,12.0,38.0,spring,8.660254e-01,-5.000000e-01,56403,60751.0,0.4160,0.096901,0.205254,22.000000
1,1,90001,2020-05-01,31.0,22.0,25.0,16.0,12.0,spring,5.000000e-01,-8.660254e-01,56403,60751.0,0.4160,0.096901,0.205254,17.666667
2,2,90001,2020-06-01,29.0,36.0,22.0,25.0,16.0,summer,1.224647e-16,-1.000000e+00,56403,60751.0,0.4160,0.096901,0.205254,21.000000
3,3,90001,2020-07-01,37.0,27.0,36.0,22.0,25.0,summer,-5.000000e-01,-8.660254e-01,56403,60751.0,0.4160,0.096901,0.205254,27.666667
4,4,90001,2020-08-01,24.0,29.0,27.0,36.0,22.0,summer,-8.660254e-01,-5.000000e-01,56403,60751.0,0.4160,0.096901,0.205254,28.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4018,6682,91801,2022-02-01,0.0,0.0,0.0,0.0,0.0,winter,8.660254e-01,5.000000e-01,53730,85549.0,0.4561,0.059655,0.114839,0.000000
4019,6683,91801,2022-03-01,0.0,0.0,0.0,0.0,0.0,spring,1.000000e+00,6.123234e-17,53730,85549.0,0.4561,0.059655,0.114839,0.000000
4020,6684,91801,2022-04-01,0.0,0.0,0.0,0.0,0.0,spring,8.660254e-01,-5.000000e-01,53730,85549.0,0.4561,0.059655,0.114839,0.000000
4021,6685,91801,2022-05-01,0.0,0.0,0.0,0.0,0.0,spring,5.000000e-01,-8.660254e-01,53730,85549.0,0.4561,0.059655,0.114839,0.000000


## Feature Selection and Data Prep

In [213]:
def prep_data(df, features, label, X_scaler, Y_scaler):
    # Shuffle
    indices = df.index.values.tolist()
    shuffled_indices = np.random.permutation(indices)
    df_shuffled = df.reindex(shuffled_indices)

    # Drop records with any nulls
    df_shuffled = df_shuffled.dropna()
    
    # Get features and label
    X = df_shuffled[features]
    Y = df_shuffled[label]

    # Scale data
    X_std = X_scaler.transform(X)
    Y_std = Y_scaler.transform(Y)

    return X_std, Y_std

In [214]:
features = ["mean_violent_prev_3mo"]#, "log_median_household_income"] #, "violent_lag_1", "pop_total", "month_sin", "month_cos"]
label = ["crime_Violent"]

# Features to scale
scale_cols = [col for col in features if col not in ["month_sin", "month_cos"]]

# Initialize scalers on training data
X_scaler = ColumnTransformer(
    transformers = [
        ("scaler", StandardScaler(), scale_cols)
        ],
    remainder = "passthrough"
    )
X_scaler.fit(df_train.dropna()[features])
Y_scaler = StandardScaler()
Y_scaler.fit(df_train.dropna()[label])

X_train, Y_train = prep_data(df_train, features, label, X_scaler, Y_scaler)
X_val, Y_val = prep_data(df_val, features, label, X_scaler, Y_scaler)
X_test, Y_test = prep_data(df_test, features, label, X_scaler, Y_scaler)

In [215]:
print("Mean:", np.mean(df_train[["crime_Violent"]]))
print("Standard deviation:", np.std(df_train[["crime_Violent"]]))

Mean: 41.39522744220731
Standard deviation: crime_Violent    47.159224
dtype: float64


/opt/miniconda3/envs/env_207_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


### Linear regression

In [216]:
tf.random.set_seed(0)

def build_model(num_features, learning_rate):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(
        units = 1, # output dim
        input_shape = [num_features], # input dim
        use_bias = True, # use a bias (intercept) param
        kernel_initializer = tf.ones_initializer, # initialize params to 1
        bias_initializer = tf.ones_initializer, # initialize bias to 1
    ))
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer = optimizer, loss = "mse")
    
    return model


model_lr = build_model(num_features = X_train.shape[1], learning_rate = 0.01)
fit_model_lr = model_lr.fit(x = X_train, y = Y_train,
                            epochs = 5,
                            validation_data = (X_val, Y_val))

Epoch 1/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 0.2646 - val_loss: 0.0624
Epoch 2/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 0.0535 - val_loss: 0.0469
Epoch 3/5
  1/119 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1020

/opt/miniconda3/envs/env_207_project/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - loss: 0.0515 - val_loss: 0.0462
Epoch 4/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 0.0515 - val_loss: 0.0461
Epoch 5/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - loss: 0.0515 - val_loss: 0.0461


In [219]:
print("Training (standardized) MSE:", model_lr.evaluate(X_train, Y_train, verbose = 0))
print("Training (standardized) RMSE:", np.sqrt(model_lr.evaluate(X_train, Y_train, verbose = 0)))

print("Validation (standardized) MSE:", model_lr.evaluate(X_val, Y_val, verbose = 0))
print("Validation (standardized) RMSE:", np.sqrt(model_lr.evaluate(X_val, Y_val, verbose = 0)))

print("Test (standardized) MSE:", model_lr.evaluate(X_test, Y_test, verbose = 0))
print("Test (standardized) RMSE:", np.sqrt(model_lr.evaluate(X_test, Y_test, verbose = 0)))

Training (standardized) MSE: 0.051464419811964035
Training (standardized) RMSE: 0.2268577082930268
Validation (standardized) MSE: 0.04610040411353111
Validation (standardized) RMSE: 0.2147100466059544
Test (standardized) MSE: 0.054993171244859695
Test (standardized) RMSE: 0.23450622858435913


In [218]:
# Revert scale and calculate MSE
def evaluate_rescale(X_std, Y_std, Y_scaler):
    Y_pred_std = model_lr.predict(X_std).reshape(-1, 1)
    Y_pred_orig = Y_scaler.inverse_transform(Y_pred_std)
    
    Y_true_std = np.array(Y_std).reshape(-1, 1)
    Y_true_orig = Y_scaler.inverse_transform(Y_true_std)
    
    mse_orig = mean_squared_error(Y_true_orig, Y_pred_orig)
    rmse_orig = np.sqrt(mse_orig)

    return mse_orig, rmse_orig

mse_train, rmse_train = evaluate_rescale(X_train, Y_train, Y_scaler)
mse_val, rmse_val = evaluate_rescale(X_val, Y_val, Y_scaler)
mse_test, rmse_test = evaluate_rescale(X_test, Y_test, Y_scaler)

print(f"Train MSE (original scale): {mse_train}")
print(f"Train RMSE (original scale): {rmse_train}")

print(f"Validation MSE (original scale): {mse_val}")
print(f"Validation RMSE (original scale): {rmse_val}")

print(f"Test MSE (original scale): {mse_test}")
print(f"Test RMSE (original scale): {rmse_test}")

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step
Train MSE (original scale): 116.51548999215548
Train RMSE (original scale): 10.79423410864131
Validation MSE (original scale): 104.37135399420208
Validation RMSE (original scale): 10.216229930566465
Test MSE (original scale): 124.50459702662107
Test RMSE (original scale): 11.158162797997754


### Random Forest

Random forest is a little better with population and the month sin/cos transformations, but a bit more overfitting (minor, but if we have generalizable models with the same validation accuracy don't think we would care about the better training performance?)

In [227]:
df_train[["violent_lag_1", "violent_lag_2", "violent_lag_3"]]

,violent_lag_1,violent_lag_2,violent_lag_3
0,16.0,12.0,38.0
1,25.0,16.0,12.0
2,22.0,25.0,16.0
3,36.0,22.0,25.0
4,27.0,36.0,22.0
...,...,...,...
4018,0.0,0.0,0.0
4019,0.0,0.0,0.0
4020,0.0,0.0,0.0
4021,0.0,0.0,0.0


In [232]:
features = ["violent_lag_1", "violent_lag_2", "violent_lag_3"]#, "month_sin", "month_cos"]
label = ["crime_Violent"]

X_train = df_train[features]
Y_train = df_train[label].values.ravel()

X_val = df_val[features]
Y_val = df_val[label].values.ravel()

X_test = df_test[features]
Y_test = df_test[label].values.ravel()

In [233]:
model_rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=1234,
    n_jobs=-1
)
model_rf.fit(X_train, Y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [237]:
Y_pred_train = model_rf.predict(X_train)
mse_train = mean_squared_error(Y_train, Y_pred_train)

Y_pred_val = model_rf.predict(X_val)
mse_val = mean_squared_error(Y_val, Y_pred_val)

Y_pred_test = model_rf.predict(X_test)
mse_test = mean_squared_error(Y_test, Y_pred_test)

print("Training (standardized) RMSE:", np.sqrt(mse_train))
print("Validation (standardized) RMSE:", np.sqrt(mse_val))
print("Test (standardized) RMSE:", np.sqrt(mse_test))

Training (standardized) RMSE: 6.775312159610979
Validation (standardized) RMSE: 10.810482196690764
Test (standardized) RMSE: 11.105835893639625


### Gradient Boosting

In [238]:
model_gb = GradientBoostingRegressor(loss="absolute_error",
                                learning_rate=0.01,
                                n_estimators=100,
                                max_depth = 10, 
                                random_state = 1234,
                                max_features = 5)
model_gb.fit(X_train, Y_train)

,loss,'absolute_error'
,learning_rate,0.01
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,10
,min_impurity_decrease,0.0
,init,None


In [239]:
Y_pred_train = model_gb.predict(X_train)
mse_train = mean_squared_error(Y_train, Y_pred_train)

Y_pred_val = model_gb.predict(X_val)
mse_val = mean_squared_error(Y_val, Y_pred_val)

Y_pred_test = model_gb.predict(X_test)
mse_test = mean_squared_error(Y_test, Y_pred_test)

print("Training (standardized) RMSE:", np.sqrt(mse_train))
print("Validation (standardized) RMSE:", np.sqrt(mse_val))
print("Test (standardized) RMSE:", np.sqrt(mse_test))

Training (standardized) RMSE: 28.737703259438863
Validation (standardized) RMSE: 29.27517778117197
Test (standardized) RMSE: 32.58078949981394


## XGBoost

In [259]:
model_xgb = XGBRegressor(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.5,
    random_state=1234,
    n_jobs=-1,
    objective="reg:squarederror")
model_xgb.fit(X_train, Y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.5
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [260]:
Y_pred_train = model_xgb.predict(X_train)
mse_train = mean_squared_error(Y_train, Y_pred_train)

Y_pred_val = model_xgb.predict(X_val)
mse_val = mean_squared_error(Y_val, Y_pred_val)

Y_pred_test = model_xgb.predict(X_test)
mse_test = mean_squared_error(Y_test, Y_pred_test)

print("Training (standardized) RMSE:", np.sqrt(mse_train))
print("Validation (standardized) RMSE:", np.sqrt(mse_val))
print("Test (standardized) RMSE:", np.sqrt(mse_test))

Training (standardized) RMSE: 8.76144913255868
Validation (standardized) RMSE: 10.871311888621051
Test (standardized) RMSE: 11.72599283951275


In [252]:
model_xgb.feature_importances_

array([0.40799782, 0.58070004, 0.01130209], dtype=float32)